# Start Up: 
Before Running the cell below, you must ensure that these have been run in Terminal **IN ORDER** : 
- conda update -n base -c defaults conda 

    - cd SageMaker
    
      - cd yelp-dataset-challenge-1-ds
      
         - conda env create -f environment.yml
          
            - source activate ydc1 
                
                - pip install python-decouple
                  
                  - pip install pprintpp

# Imports: 

In [1]:
import pandas as pd
import s3
from pprintpp import pprint as pp
from sklearn.externals import joblib
import json

# Load in Bucket
bucket = s3.Bucket('yelpchallenge1')

# Look inside Bucket 
bucket.contents

['API/',
 'API/api.py',
 'API/api_exploration.ipynb',
 'Environments/',
 'Environments/environment.yml',
 'Flask_App/',
 'Flask_App/Pipfile',
 'Flask_App/__init__.py',
 'Flask_App/app.py',
 'Flask_App/models.py',
 'Flask_App/yelp.py',
 'Model/',
 'Model/vect_1.sav',
 'datasets/',
 'datasets/df.csv',
 'datasets/dtm.csv',
 'datasets/dtm_final.csv',
 'datasets/review.json',
 'datasets/user.json',
 'notebooks/',
 'notebooks/data_cleanup.ipynb',
 'notebooks/official_NB.ipynb',
 'notebooks/vectorization_exploration.ipynb',
 'notebooks/yelp_data_initial_exploration.ipynb']

In [ ]:
                    ### ***** DO NOT RUN. ******* #### 
                  ### ***** ALREADY INSTALLED. ****** ###

    # Installs the File 'Locally' on SageMaker Instance / Only have to run these once: 

#bucket.get('datasets/df.csv', 'df.csv')

    # Installing .json Files 'Locally'
    
#bucket.get('datasets/user.json', 'user.json')
#bucket.get('datasets/review.json', 'review.json')

# Cleaning Data: Complete as of ***8:14 PM : 12/19/2019***

Cleaning df.csv & saving Cleaned df.csv

In [20]:
                    ### ***** DO NOT RUN. ******* #### 
                  ### ***** ALREADY COMPLETE. ****** ###
# Further Cleaning of df.csv: 

# Dropping Column:
#df = df.drop(columns=['Unnamed: 0'])

# Dropping all Missing / Na Values from Entire Dataframe:
#df = df.dropna()

    # Saving Cleaned df.csv 
#df.to_csv(index=False)
#df.to_csv(r'df.csv')

Converting user_json & to Pandas DataFrame / Saving as user.csv & review.csv

In [21]:
                              # ******* DO NOT RUN! ******* # 
                            # ***** ALREADY COMPLETE. ****** # 
 # import user.json
#with open('user.json') as f:
    #user = json.loads("[" + 
                      #f.read().replace("}\n{", "},\n{") + 
                      #"]")
    
    # convert user.json files to pandas DataFrame 'user_df'
#user_df = pd.DataFrame(user)

    # Saving user_df as csv file. 
#user_df.to_csv(index=False)
#user_df.to_csv(r'user.csv')

# Import review.json 
#with open('review.json') as f:
    #review = json.loads("[" + 
                       # f.read().replace("}\n{", "},\n{") + 
                        #"]")
    
    # convert review.json files to pandas DataFrame 'review_df'
#review_df = pd.DataFrame(review)

    # Saving user_df as csv file. 
#review_df.to_csv(index=False)
#review_df.to_csv(r'review.csv')

# Data Merging: Complete as of ***8:14 PM : 12/19/2019***

In [ ]:
            # ***** New DTM DF HAS BEEN CREATED. DO NOT RUN THIS CELL **** #
# Read-in dtm.csv (Original)
#dtm = pd.read_csv('dtm.csv')

    # Taking Stars Column
#stars = df['stars']

    # Adding stars column to dtm
#dtm['stars']=df['stars']

# Shifting 'Stars' Column to front of Df,
#cols = list(dtm.columns)
#cols = [cols[-1]] + cols[:-1]
#dtm = dtm[cols]

    # Dropping "-PRON-", 'year -PRON-', and ' ' Columns
#dtm = dtm.drop(columns=[' ', '  -PRON-', 'year -PRON-'])#Cut 135,000 Rows of df['stars'] Column to fix Memory Error. 
    # Label as "stars"
#stars = df.stars[0:135000]
#stars.shape
    # Adding stars to dtm2
#dtm2['stars']=df['stars'][0:135000]

In [ ]:
             # ***** New DTM2 DF HAS BEEN CREATED. DO NOT RUN THIS CELL **** #
    
    # Read-in dtm2.csv(Old)
#dtm2 = pd.read_csv('dtm2.csv')

    # Taking Stars Column
#stars = df['stars']

    # Adding stars column to dtm
#dtm2['stars']=df['stars']

    # Shifting 'Stars' Column to front of Df,
#cols = list(dtm2.columns)
#cols = [cols[-1]] + cols[:-1]
#dtm2 = dtm2[cols]

#dtm2 = dtm2.drop(columns=['stars'])

    # Dropping columns: 
#dtm2 = dtm2.drop(columns=[' ' , '  '])
#dtm2 = dtm2.drop(columns=['  -PRON-','  i',  '  the',  '  this', '$', "'s"])
    # Saving dtm2.csv 
#dtm2.to_csv(index=False)
#dtm2.to_csv(r'dtm2.csv')

    # Cut 135,000 Rows of df['stars'] Column to fix Memory Error. 
        # Label as "stars"     
#stars = df.stars[0:135000]
#stars.shape

        # Adding stars to dtm2
#dtm2['stars']=df['stars'][0:135000]

    # Saving Final df as 'dtm_final'
#dtm_final = dtm2

    # Saving dtm_final.csv 
#dtm_final.to_csv(index=False)
#dtm_final.to_csv(r'dtm_final.csv')


In [ ]:
# Read-in dtm_final.csv (FINAL)
#dtm_final = pd.read_csv('dtm_final.csv')

# Clean / Analyze user.csv & review.csv: 

In [21]:
# Imports

    # Read-in user.csv
user = pd.read_csv('user.csv')
    # Read-in review.csv
review = pd.read_csv('review.csv')

In [22]:
# Check Read-in of df_user
# Checking Null Values and Shape
pp(user.isna().sum())
pp(user.shape)

Unnamed: 0                  0
average_stars               0
compliment_cool             0
compliment_cute             0
compliment_funny            0
compliment_hot              0
compliment_list             0
compliment_more             0
compliment_note             0
compliment_photos           0
compliment_plain            0
compliment_profile          0
compliment_writer           0
cool                        0
elite                 1565761
fans                        0
friends                     0
funny                       0
name                        3
review_count                0
useful                      0
user_id                     0
yelping_since               0
dtype: int64
(1637138, 23)


Three Problems: 
**Problem 1:**
user['Unamed: 0'] should not exist. 

**Problem 2:**
user['elite'] has 1,565,761 Missing Values. 

**Problem 3:** 
user['name'] has 3 Missing Values. 

**Solution?:**
Drop user['Unamed: 0' , 'elite' , 'name']  Columns.

In [ ]:
# Solution:
user_df = user.drop(columns=['Unnamed: 0', 'elite' , 'name'])

# Save Cleaned user_df.csv 
user_df.to_csv(index=False)
user_df.to_csv(r'user.csv')